In [25]:
import glob
import os
import io
import json
import shutil
from PIL import Image


img_dir='/home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/images/'
bboxes_dir='/home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/bboxes/'


images_folders=os.listdir(img_dir)[:10]

images_paths = []
coordinates=[]
classes=[]


#convert images,coordinates and classes to a yolo dataset format
def to_yolo(images_paths,coordinates,classes,yolo_folder):
    for image in images_paths:
        shutil.copy(image,yolo_folder)
    for i in range(len(images_paths)):
        with io.open(yolo_folder+'/'+images_paths[i].split('/')[-1].split('.')[0]+'.txt','w',encoding='utf8') as f:
            for j in range(len(coordinates[i])):
                f.write(str(classes[i][j])+' '+str(coordinates[i][j][0])+' '+str(coordinates[i][j][1])+' '+str(coordinates[i][j][2])+' '+str(coordinates[i][j][3])+'\n')

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

# labels_dict={'title':1,'paragraph':2,'picture':3,'table':4,'formula':5,'footnote':6,'page-footer':7,'page-header':8,'subtitle-level-1':9,'caption':10}
labels_dict={'caption':0,'footnote':1,'formula':2,'page-footer':3,'page-header':4,'paragraph':5,'picture':6,'subtitle-level-1':7,'table':8,'title':9}
# labels_dict={'caption':1,'footnote':2,'formula':3,'page-footer':4,'page-header':5,'paragraph':6,'picture':7,'subtitle-level-1':8,'table':9,'title':10}

resolution=150
base_path='/home/kiki/dados_servidor/Servidor/Teses/labeling/dataset/'
for folder in images_folders:
    
    annotations=json.load(open((bboxes_dir+folder+'.json')))
    #mkdir
    if not os.path.exists(base_path+folder):
        os.mkdir(base_path+folder)
    if not os.path.exists(base_path+folder+'/obj_train_data'):
        os.mkdir(base_path+folder+'/obj_train_data')
    for annotation in annotations.keys():
        img_path=img_dir+folder+'/'+folder+'_'+annotation+'.jpg'
        images_paths.append(img_path)
        im=Image.open(img_path)
        w= int(im.size[0])
        h= int(im.size[1])
        bboxes=[an['bbox'] for an in annotations[annotation]]
        bboxes = [[x * resolution / 72 for x in b] for b in bboxes]
        for i in range(len(bboxes)):
            bbox=bboxes[i]
            x_min,y_min,x_max,y_max=int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])
            #convert to yolo format (x_center,y_center,width,height)
            bbox=convert((w,h),(x_min,x_max,y_min,y_max))
            bboxes[i]=bbox
        
        coordinates.append(bboxes)
        # classes.append([an['type'] for an in annotations[annotation]])
        classes.append([labels_dict[an['type']] for an in annotations[annotation]])
    yolo_folder=base_path+folder+'/obj_train_data'
    to_yolo(images_paths,coordinates,classes,yolo_folder)

    p='/home/kiki/dados_servidor/Servidor/Teses/labeling/dataset/'+folder+'/'
    #obj names
    with io.open(p+'obj.names','w',encoding='utf8') as f:
        for key in labels_dict.keys():
            f.write(key.replace('-','_')+'\n')
    #train txt
    with io.open(p+'train.txt','w',encoding='utf8') as f:
        for image in images_paths:
            f.write('data/obj_train_data/'+image.split('/')[-1]+'\n')
    #obj data
    with io.open(p+'obj.data','w',encoding='utf8') as f:
        f.write('classes = '+str(len(labels_dict.keys()))+'\n')
        # f.write('classes = '+str(len(labels_dict.keys())+1)+'\n')
        f.write('train = data/train.txt\n')
        # f.write('valid = data/train.txt\n')
        f.write('names = data/obj.names\n')
        f.write('backup = backup/\n')